# Overview of some new Python 3 features
This notebook shows some new Python 3 features. Some of them have been backported to Python 2 in the mean time, but you might not know them yet :-)

## Advanced string formatting (backported to Python 2)

In [1]:
print('Hello {}!'.format('world'))

Hello world!


In [2]:
values = {'first_name': 'John', 'last_name': 'Doe'}
print('Hello {first_name} {last_name}!'.format(**values))

Hello John Doe!


In [3]:
for n in range(11):
    print('2^{0:<2} = {1:4} (in hex: 2^{0:<x} = {1:3x})'.format(n, 2**n))

2^0  =    1 (in hex: 2^0 =   1)
2^1  =    2 (in hex: 2^1 =   2)
2^2  =    4 (in hex: 2^2 =   4)
2^3  =    8 (in hex: 2^3 =   8)
2^4  =   16 (in hex: 2^4 =  10)
2^5  =   32 (in hex: 2^5 =  20)
2^6  =   64 (in hex: 2^6 =  40)
2^7  =  128 (in hex: 2^7 =  80)
2^8  =  256 (in hex: 2^8 = 100)
2^9  =  512 (in hex: 2^9 = 200)
2^10 = 1024 (in hex: 2^a = 400)


## Stuff that you can do with `*` 

In [4]:
# Known from Python 2: handling positional and keyword function arguments
def f(*args, **kwargs):
    print("f called with args={}, kwargs={}".format(args, kwargs))

def g(*args, **kwargs):
    f(*args, **kwargs)

g(1, 2, a=3, b=4)

f called with args=(1, 2), kwargs={'a': 3, 'b': 4}


In [5]:
# New: extended iterable unpacking
a, b, *rest = range(5)
print("a = {}, b = {}, rest = {}".format(a, b, rest))

a = 0, b = 1, rest = [2, 3, 4]


In [6]:
# New: syntax for keyword-only arguments
def f(a, b, *, c):
    print(a, b, c)

f(1, 2, c=3)
#f(1, 2, 3) # does not work because c is a keyword-only argument

1 2 3


In [7]:
# New: multiple unpackings in function calls
args1 = (1, 2)
args2 = (3, 4)
kwargs1 = {'e': 5}
kwargs2 = {'f': 6}

def f(a, b, c, d, e, f):
    print(a, b, c, d, e, f)

f(*args1, *args2, **kwargs1, **kwargs2)

1 2 3 4 5 6


In [8]:
# New: unpackings in other contexts
t = *range(4), 4
print(type(t), t)
l = [*range(4), 4]
print(type(l), l)

<class 'tuple'> (0, 1, 2, 3, 4)
<class 'list'> [0, 1, 2, 3, 4]


## Sets and dictionaries

In [9]:
# Set literals (backported to Python 2)
s = {1, 2, 3, 5, 8}
print(type(s), s)

<class 'set'> {1, 2, 3, 5, 8}


In [10]:
# Dict and set comprehensions (backported to Python 2)
d = {x: 2*x for x in range(5)}
s = {2*x for x in range(3)}
print("d =", d)
print("s =", s)

d = {0: 0, 1: 2, 2: 4, 3: 6, 4: 8}
s = {0, 2, 4}


In [11]:
# Merging dictionaries
d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4}
{**d1, **d2}

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

## Unicode: clear separation between Unicode objects (`str`) and `bytes`

In [12]:
b = b'abc'
s = 'abc'
print(type(b), type(s), b == s)

<class 'bytes'> <class 'str'> False


In [13]:
s = 'Viele Grüße!'
print(s)
print(s.encode('utf-8'))
print(s.encode()) # utf-8 is default
print(s.encode('utf-16'))
print(s.encode('iso-8859-1'))

Viele Grüße!
b'Viele Gr\xc3\xbc\xc3\x9fe!'
b'Viele Gr\xc3\xbc\xc3\x9fe!'
b'\xff\xfeV\x00i\x00e\x00l\x00e\x00 \x00G\x00r\x00\xfc\x00\xdf\x00e\x00!\x00'
b'Viele Gr\xfc\xdfe!'


## Easy caching of function results
Note that the Fibonacci function is only used for illustration purposes here. Calculating it iteratively is faster and more Pythonic.

In [14]:
%%time

# Slow recursive calculation of Fibonacci numbers
def fib_slow(n):
    if n <= 1:
        return 1
    else:
        return fib_slow(n - 1) + fib_slow(n - 2)

print(" ".join(str(fib_slow(n)) for n in range(32)))
print()

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309

CPU times: user 1.59 s, sys: 7.94 ms, total: 1.59 s
Wall time: 1.64 s


In [15]:
%%time

import functools

# Fast recursive calculation of Fibonacci numbers - the only change is the decorator!
@functools.lru_cache(maxsize=None)
def fib_fast(n):
    if n <= 1:
        return 1
    else:
        return fib_fast(n - 1) + fib_fast(n - 2)

print(" ".join(str(fib_fast(n)) for n in range(32)))
print()

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309

CPU times: user 121 µs, sys: 5 µs, total: 126 µs
Wall time: 131 µs


## Multiple context managers in a single `with` statement

In [16]:
# Python 2&3
with open("file1.txt", "w") as f1:
    with open("file2.txt", "w") as f2:
        for i in range(5):
            print(i, file=f1, end=' ')
            print(2*i, file=f2, end=' ')

In [17]:
# Python 3
with open("file1.txt", "w") as f1, open("file2.txt", "w") as f2:
    for i in range(5):
        print(i, file=f1, end=' ')
        print(2*i, file=f2, end=' ')

In [18]:
!cat file1.txt; echo; cat file2.txt

0 1 2 3 4 
0 2 4 6 8 

## New classes in `collections` (backported to Python 2)

In [19]:
# Iteration order is undefined for dict
d = {"ABCDE"[n]: n for n in range(5)}
tuple(d.items())

(('A', 0), ('B', 1), ('C', 2), ('D', 3), ('E', 4))

In [20]:
# Insertion order is preserved for OrderedDict
import collections

od = collections.OrderedDict(("ABCDE"[n], n) for n in range(5))
tuple(od.items())

(('A', 0), ('B', 1), ('C', 2), ('D', 3), ('E', 4))

In [21]:
# Counting objects
import collections
counts = collections.Counter("the quick brown fox jumps over the lazy dog")
print('The letter {} occurs {} times'.format('e', counts['e']))
print('The most common letters are:', counts.most_common(5))

The letter e occurs 3 times
The most common letters are: [(' ', 8), ('o', 4), ('e', 3), ('t', 2), ('h', 2)]


## Delegating to a subgenerator with `yield from`

In [22]:
# Nested list of ints as a simple example for a tree structure
l = [[1, 2, [3, 4]], 5, [[6, 7], 8]]

In [23]:
# Python 2&3:
def walk2(tree):
    if isinstance(tree, int):
        yield tree
    else:
        for subtree in tree:
            for item in walk2(subtree):
                yield item
                
print(" ".join(str(n) for n in walk2(l)))

1 2 3 4 5 6 7 8


In [24]:
# Python 3:
def walk3(tree):
    if isinstance(tree, int):
        yield tree
    else:
        for subtree in tree:
            yield from walk3(subtree)
                
print(" ".join(str(n) for n in walk3(l)))

1 2 3 4 5 6 7 8


Note that using `yield from` has many more potential benefits than this!
* Transparent handling of values sent to the delegating generator with `send()`
* Automatic propagation of exceptions (except for `StopIteration`) to the `throw()` method of the iterator
* ...

## Support for enumerations

In [25]:
import enum

class Color(enum.Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

In [26]:
# Human-readable string representations, and more info in the repr
print(Color.RED)
print(repr(Color.GREEN))

Color.RED
<Color.GREEN: 2>


In [27]:
# Item name is easily accessible
print(Color.RED.name)

RED


In [28]:
# Iteration
print(" ".join(str(c) for c in Color))

Color.RED Color.GREEN Color.BLUE


In [29]:
# Enum members are hashable, i.e., usable as keys in dict and set
{color: str(color) for color in Color}

{<Color.RED: 1>: 'Color.RED',
 <Color.GREEN: 2>: 'Color.GREEN',
 <Color.BLUE: 3>: 'Color.BLUE'}

In [30]:
# Converting indices to enum members
[Color(i) for i in range(1, 4)]

[<Color.RED: 1>, <Color.GREEN: 2>, <Color.BLUE: 3>]

In [31]:
# Creation of enumerations with the functional API
Animal = enum.Enum('Animal', ('ANT', 'BEE', 'CAT', 'DOG'))
tuple(Animal)

(<Animal.ANT: 1>, <Animal.BEE: 2>, <Animal.CAT: 3>, <Animal.DOG: 4>)

Note that the class `enum` has even more useful features!

## Type hints

In [32]:
def greeting(name: str) -> str:
    return 'Hello {}'.format(name)

print(greeting("friend"))
print(greeting(5)) # No runtime type checking!

Hello friend
Hello 5


In [33]:
from typing import List, Dict, Tuple
def get_items(d: Dict[int, str]) -> List[Tuple[int, str]]:
    return tuple(d.items())

get_items({1: 'A', 2: 'B'})

((1, 'A'), (2, 'B'))

## Coroutines with `async` and `await` syntax

In [34]:
import asyncio
import datetime

def now():
    return datetime.datetime.now().time()

async def request(seconds, name):
    print("{}: waiting for {} seconds before processing request {}".format(now(), seconds, name))
    await asyncio.sleep(seconds) # <-- this could be any time-consuming IO operation
    print("{}: processed request {}".format(now(), name))
    return name

async def multiple_requests(count):
    print("{}: sending {} requests".format(now(), count))
    result = await asyncio.gather(*(request(count - n, n) for n in range(count)))
    print("{}: all responses have been collected".format(now()))
    return result

async def main():
    result = await multiple_requests(5)
    print("{}: result is {}".format(now(), result))

# Use this in a normal Python script if an event loop is already running:
#
# loop = asyncio.get_event_loop()
# loop.run_until_complete(main())
#
# Use this to start an event loop and execute main() in it:
#
# asyncio.run(main())
#
# In a Jupyter notebook, there is already a running event loop. Therefore, we can simply await the coroutine:

await main()

16:11:50.429893: sending 5 requests
16:11:50.430152: waiting for 5 seconds before processing request 0
16:11:50.430297: waiting for 4 seconds before processing request 1
16:11:50.430357: waiting for 3 seconds before processing request 2
16:11:50.430396: waiting for 2 seconds before processing request 3
16:11:50.430432: waiting for 1 seconds before processing request 4
16:11:51.431665: processed request 4
16:11:52.431444: processed request 3
16:11:53.431840: processed request 2
16:11:54.432200: processed request 1
16:11:55.431652: processed request 0
16:11:55.432144: all responses have been collected
16:11:55.432188: result is [0, 1, 2, 3, 4]
